## Camera-Based Detections

This script will show the process for training, converting, and quantizing the camera-based
detection portion of the project. 

## Import Required Dependencies

In [18]:
from parameters import Parameters
from tflite import TFliteRunner
import tensorflow as tf
import shutil
import glob
import yaml
import cv2
import os

## Format Dataset

The dataset can be downloaded from this link:
https://universe.roboflow.com/vishwakarma-institute-of-technology-yqqb5/edi-ty/dataset/14/download

1) Download the dataset following these options:

![Dataset Download Options](images/dataset_download.png)

2) Extract the dataset.

3) Run the cells below to reformat the dataset, ensure the paths are correct.

This cells will reformat the dataset to be used for training and validation.

In [6]:
# The path pointing to the extracted dataset.
extracted_dataset_path = "EDI TY.v14i.yolov5pytorch"
if not os.path.exists(extracted_dataset_path):
    raise FileNotFoundError(f"The dataset is not found at: {extracted_dataset_path}")

# The path to save the formatted dataset.
formatted_dataset_path = "RoadDetectionDataset_v0"
if not os.path.exists(formatted_dataset_path):
    os.makedirs(formatted_dataset_path)

# Create images subdirectories
os.makedirs(os.path.join(formatted_dataset_path, "images/train"))
os.makedirs(os.path.join(formatted_dataset_path, "images/validate"))
os.makedirs(os.path.join(formatted_dataset_path, "images/test"))
# Create labels subdirectories
os.makedirs(os.path.join(formatted_dataset_path, "labels/train"))
os.makedirs(os.path.join(formatted_dataset_path, "labels/validate"))
os.makedirs(os.path.join(formatted_dataset_path, "labels/test"))
# Create dataset.yaml file
contents = {
    "path": os.path.abspath(formatted_dataset_path),
    "train": os.path.abspath(os.path.join(formatted_dataset_path, "images/train")),
    "val": os.path.abspath(os.path.join(formatted_dataset_path, "images/validate")),
    "test": os.path.abspath(os.path.join(formatted_dataset_path, "images/test")),
    "names": {
        0: "bike",
        1: "car",
        2: "cycle",
        3: "pedestrian",
        4: "signal"
    }
}
with open(os.path.join(formatted_dataset_path, "dataset.yaml"), 'w') as outfile:
    yaml.dump(contents, outfile, default_flow_style=False)

with open(os.path.join(formatted_dataset_path, "labels.txt"), "w") as file:
    file.write("bike\n")
    file.write("car\n")
    file.write("cycle\n")
    file.write("pedestrian\n")
    file.write("signal")

In [12]:
train_images = glob.glob(os.path.join(extracted_dataset_path, "train/images/*.jpg"))
validate_images = glob.glob(os.path.join(extracted_dataset_path, "valid/images/*.jpg"))
test_images = glob.glob(os.path.join(extracted_dataset_path, "test/images/*.jpg"))
print(f"{len(train_images)=}, {len(validate_images)=}, {len(test_images)=}")

train_labels = glob.glob(os.path.join(extracted_dataset_path, "train/labels/*.txt"))
validate_labels = glob.glob(os.path.join(extracted_dataset_path, "valid/labels/*.txt"))
test_labels = glob.glob(os.path.join(extracted_dataset_path, "test/labels/*.txt"))
print(f"{len(train_labels)=}, {len(validate_labels)=}, {len(test_labels)=}")

# Move images
for i, train_image in enumerate(train_images):
    print(f"Moving training image {i}", end="\r")
    new_path = os.path.abspath(os.path.join(formatted_dataset_path, f"images/train/{os.path.basename(train_image)}"))
    shutil.move(train_image, new_path)

for i, validate_image in enumerate(validate_images):
    print(f"Moving validate image {i}", end="\r")
    new_path = os.path.abspath(os.path.join(formatted_dataset_path, f"images/validate/{os.path.basename(validate_image)}"))
    shutil.move(validate_image, new_path)

for i, test_image in enumerate(test_images):
    print(f"Moving test image {i}", end="\r")
    new_path = os.path.abspath(os.path.join(formatted_dataset_path, f"images/test/{os.path.basename(test_image)}"))
    shutil.move(test_image, new_path)

# Move annotations
for i, train_label in enumerate(train_labels):
    print(f"Moving training label {i}", end="\r")
    new_path = os.path.abspath(os.path.join(formatted_dataset_path, f"labels/train/{os.path.basename(train_label)}"))
    shutil.move(train_label, new_path)

for i, validate_label in enumerate(validate_labels):
    print(f"Moving validate label {i}", end="\r")
    new_path = os.path.abspath(os.path.join(formatted_dataset_path, f"labels/validate/{os.path.basename(validate_label)}"))
    shutil.move(validate_label, new_path)

for i, test_label in enumerate(test_labels):
    print(f"Moving test label {i}", end="\r")
    new_path = os.path.abspath(os.path.join(formatted_dataset_path, f"labels/test/{os.path.basename(test_label)}"))
    shutil.move(test_label, new_path)

len(train_images)=2801, len(validate_images)=802, len(test_images)=397
len(train_labels)=2801, len(validate_labels)=802, len(test_labels)=397


## Train and Convert the Model

This portion of the project utilizes [YoloV5](https://github.com/ultralytics/yolov5). 
1) The repository can be cloned using this command.

    ```shell
    git clone https://github.com/ultralytics/yolov5.git
    ```

2) Once the repository is cloned, go to the repository and with a terminal, start
the training process with the following command:

    *Ensure to change the path to point to the dataset.yaml that is inside the formatted dataset.*

    ```shell
    python train.py --data=<path to the dataset.yaml> --epochs=20
    ```

    *After training, the model files will be found under `/runs/train/exp<n>`*

3) The next step is to perform conversion of the model to TFLite and TensorFlow saved model using this command.

    ```shell
    python export.py --data=<path to the dataset.yaml> --weights=<path to the trained .pt model> --keras --include=["saved_model", "tflite"]
    ```

4) The next step is to take the float16 saved model and quantize it to Int8.

## Post Training Dynamic Quantization of the Model

The cell below performs post training dynamic quantization of the float16 Tensorflow Saved Model into Int8 TFLite model.

After conversion, YoloV5 will output a tensorflow saved_model with the directory structure shown below:

```
saved_model-keras
        |---------- assets
        |---------- variables
        |---------- fingerprint.pb
        |---------- keras_metadata.pb
        |---------- saved_model.pb
```

In [ ]:
# Change the path to point to the saved_model directory.
converter = tf.lite.TFLiteConverter.from_saved_model("epoch-20-BMPCS-best_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Save TFLite Model
with open('epoch-20-BMPCS-best-BMPCS-_quantized_saved_model.tflite', 'wb') as f:
    f.write(tflite_quant_model)

## Validation

1) Validation of the models utilizes `deepview-validator` which can be installed using

    ```shell
    pip install deepview-validator
    ```

2) Due to current limitations in the validator software, move the `dataset.yaml` outside the dataset directory for now.

3) This is the command template used to validate the models once validator is installed.

    ```shell
    deepview-validator <path to the tflite model> --dataset=<path to the formatted dataset> --norm=unsigned --visualize=<path to save the visualization of results and metrics>
    ```

*Note: Sample commands are provided below.*

### Int Model

```shell
deepview-validator epoch-20-BMPCS-best-BMPCS-_quantized_saved_model.tflite --dataset=RoadDetectionDataset_v0 --norm=unsigned --visualize=results
```

### Float Model

```shell
deepview-validator epoch-20-BMPCS-best-fp16.tflite --dataset=RoadDetectionDataset_v0 --norm=unsigned --visualize=results
```

## Demo

Here is a demo for the camera based detection. A video is also available here.

In [35]:
# Model parameters
parameters = Parameters(
        detection_score=0.001,
        detection_iou=0.60,
        acceptance_score=0.40,
        max_detections=300,
        normalization="unsigned",
        warmup=0
    )
# Model runner
loaded_model = TFliteRunner(
    model="epoch-20-BMPCS-best-fp16.tflite",
    parameters=parameters
)

In [42]:
from utils import visualize

# define a video capture object 
vid = cv2.VideoCapture(0) 
  
while(True): 
      
    # Capture the video frame by frame 
    ret, frame = vid.read() 
  
    boxes, scores, labels = loaded_model.detect(frame)
    frame = visualize(frame, boxes, scores, labels)
    
    # Display the resulting frame 
    cv2.imshow('frame', frame) 

    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 